# Psychoinformatics - Week 11 (Exercises)
by 徐舒庭 (b11705018@ntu.edu.tw)

In [7]:
%config IPCompleter.greedy=True
%matplotlib inline

from numpy import *
from matplotlib.pyplot import *
from IPython.display import *
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
from sklearn import *

iris = datasets.load_iris()
X=iris.data; Y=iris.target

## 1 Performance Tuning of a Neural Net (8 points)

### 1.0 Baseline Performance
SVM can reach an classifcation accuracy ~ 8x% correct for the HARD Iris problem.

In [8]:
sss=model_selection.StratifiedShuffleSplit(n_splits=3,test_size=0.1) # (45+45+45) vs. (5+5+5)
model=svm.SVC(C=10)
acc=[]
for train_index, test_index in sss.split(X, Y): # 3-fold cross-validation
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    model.fit(X_train[:,0:2],Y_train) #training
    acc.append(model.predict(X_test[:,0:2])==Y_test) # testing
print(np.mean(acc))

0.8666666666666667


### 1.1 Tuning your ANN (4 points)
Tune your model hyperparameters (# of layers, # of units in each layer, activation function, optimizer, epochs, batch_size, etc.) to see if you can push your ANN performance up to ~9x% correct for the HARD iris problem.

In [125]:
from keras.models import Sequential,clone_model
from keras.layers import Dense
from keras.utils import to_categorical
import tensorflow as tf
from keras.layers import Dropout
from keras import regularizers
from keras.callbacks import LearningRateScheduler

def lr_schedule(epoch):
    return 0.1 * (0.6 ** int(epoch / 15))

optimizer = tf.keras.optimizers.legacy.SGD(lr=0.02)  # Adjust the learning rate based on experimentation

model = Sequential()
model.add(Dense(units=48, activation='relu', input_dim=2, kernel_regularizer=regularizers.l2(0.02)))  # Adjust units based on experimentation
model.add(Dropout(0.5))
model.add(Dense(units=40, activation='relu'))
model.add(Dense(units=3, activation='softmax'))


acc=[]
for train_index, test_index in sss.split(X, Y): # 3-fold cross-validation
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    new_model=clone_model(model) # Otherwise the old model will keep learning
    new_model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    new_model.fit(X_train[:, 0:2], to_categorical(Y_train), epochs=55, batch_size=15, callbacks=[LearningRateScheduler(lr_schedule)])
    acc.append(np.mean(argmax(new_model.predict(X_test[:,0:2]),1)==Y_test)) # testing
print(acc, np.mean(acc))

Epoch 1/55
9/9 [==============================] - 0s 635us/step - loss: 1.5587 - accuracy: 0.2889 - lr: 0.1000
Epoch 2/55
9/9 [==============================] - 0s 545us/step - loss: 1.1276 - accuracy: 0.3852 - lr: 0.1000
Epoch 3/55
9/9 [==============================] - 0s 559us/step - loss: 1.0199 - accuracy: 0.5852 - lr: 0.1000
Epoch 4/55
9/9 [==============================] - 0s 522us/step - loss: 0.9939 - accuracy: 0.4963 - lr: 0.1000
Epoch 5/55
9/9 [==============================] - 0s 505us/step - loss: 1.0855 - accuracy: 0.4519 - lr: 0.1000
Epoch 6/55
9/9 [==============================] - 0s 521us/step - loss: 1.0083 - accuracy: 0.5037 - lr: 0.1000
Epoch 7/55
9/9 [==============================] - 0s 501us/step - loss: 0.9777 - accuracy: 0.5037 - lr: 0.1000
Epoch 8/55
1/9 [==>...........................] - ETA: 0s - loss: 0.9728 - accuracy: 0.6000

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


9/9 [==============================] - 0s 602us/step - loss: 0.9062 - accuracy: 0.5259 - lr: 0.1000
Epoch 9/55
9/9 [==============================] - 0s 557us/step - loss: 0.9876 - accuracy: 0.4667 - lr: 0.1000
Epoch 10/55
9/9 [==============================] - 0s 537us/step - loss: 0.9071 - accuracy: 0.5704 - lr: 0.1000
Epoch 11/55
9/9 [==============================] - 0s 422us/step - loss: 0.9165 - accuracy: 0.5556 - lr: 0.1000
Epoch 12/55
9/9 [==============================] - 0s 493us/step - loss: 1.0487 - accuracy: 0.4519 - lr: 0.1000
Epoch 13/55
9/9 [==============================] - 0s 499us/step - loss: 0.9708 - accuracy: 0.5556 - lr: 0.1000
Epoch 14/55
9/9 [==============================] - 0s 425us/step - loss: 0.9821 - accuracy: 0.5037 - lr: 0.1000
Epoch 15/55
9/9 [==============================] - 0s 471us/step - loss: 0.8723 - accuracy: 0.5704 - lr: 0.1000
Epoch 16/55
9/9 [==============================] - 0s 443us/step - loss: 0.9586 - accuracy: 0.6444 - lr: 0.0600
Epoch

### 1.2 Is your (deep) network better than SVM? Why or why not? (4 points)

After tuning model hyperparameters with different combinations, I observe that the accuracy of my network is only between 6x% to 8x%. Hence, my network is not better than SVM.

Following explanations may be the reasons that my network is not better than the SVM:

1. Data Size and Complexity:
   
   The Iris dataset is small, consisting of only 150 samples. Deep networks often require large amounts of data to effectively learn complex patterns. If the dataset is not sufficiently complex, a simpler model like SVM may generalize better.

2. Model Complexity & Overfitting:
   
   The complexity of my network might be excessive for the simplicity of the Iris dataset. Increasing the number of layers and units might lead to overfitting, especially when the dataset is small.

3. Hyperparameter Tuning:
   
   Maybe there will be a combination of the hyperparameters which makes the accuracy higher than SVM, but through my experiences for over hundredes of times, I still haven't found it.

4. Regularization and Dropout:
   
   The use of regularization (e.g., L2 regularization) and dropout in my network may not be suitable for the given problem. Sometimes, too much regularization can hinder the learning process.